In [110]:
from scipy.stats import norm
from scipy.linalg import cho_factor, cho_solve, cholesky, sqrtm, det
from scipy.linalg import norm as la_norm
import numpy as np

from gproc.generative import sample_at_x
from gproc.kernels import squared_exponential

In [84]:
N = 50

# x = np.random.uniform(-1, 1, N).reshape(-1, 1) # Reshape to N x 1 matrix
x = np.linspace(-1, 1, N).reshape(-1, 1) # Reshape to N x 1 matrix
y, prob_y, f = sample_at_x(x, kernel_params={'lengthscale': 0.1, 'variance': 1.4})

model_lengthscale = 0.1
gram = squared_exponential(x, x, lengthscale = model_lengthscale)

In [85]:
JITTER = 1e-5 

In [154]:
def expectation_propagation_probit(observed_y, gram, max_iterations=100, tol=1e-5):
    """Algorithm 3.5"""
    
    # Data size and gram matrix relabelling
    N = y.shape[0]
    K = np.copy(gram)
    
    # Initialise natural site and cavity parameters
    nu_site = np.zeros(N)
    tau_site = np.zeros(N)
    
    nu_cav = np.zeros(N)
    tau_cav = np.zeros(N)
    
    z = np.zeros(N)
    
    # Initialise approximate posterior parameters
    Sig = np.copy(gram)
    mu_proposed = np.zeros(N)
    
    for i in range(1, max_iterations):
        mu = mu_proposed
        
        for i in range(N):
            # Compute approximate cavity parameters
            tau_cav[i] = Sig[i, i]**(-1) - tau_site[i]
            nu_cav[i] = Sig[i, i]**(-1) * mu[i] - nu_site[i]

            # Compute marginal moments
            z[i] = ( y[i] * nu_cav[i] / tau_cav[i] ) / (1 + tau_cav[i]**(-1))
            print(1 + tau_cav[i]**(-1))
            mu_hat = ( nu_cav[i] / tau_cav[i] ) + ( y[i] * tau_cav[i]**(-1) * norm.pdf(z[i]) ) / ( norm.cdf(z[i]) * np.sqrt( 1 + tau_cav[i]**(-1) ) ) 
            var_hat = tau_cav[i]**(-1)  - ( ( tau_cav[i]**(-2) * norm.pdf(z[i]) ) / ( ( 1 + tau_cav[i]**(-1) ) * norm.cdf(z[i]) ) ) * ( z[i] + ( norm.pdf(z[i]) / norm.cdf(z[i]) ) ) 

            # Update site parameters
            tau_delta = var_hat**(-1) - tau_cav[i] - tau_site[i]
            tau_site[i] = tau_site[i] + tau_delta

            nu_site[i] = var_hat**(-1) * mu_hat - nu_cav[i]

            # Update approximate posterior parameters
            Sig = Sig - ( ( tau_delta**(-1) + Sig[i, i] )**(-1) * Sig[:, i].dot(Sig[:, i]) )
            mu = Sig.dot(nu_site)

        # Recompute the approximate posterior parameters 
        S = np.diag(tau_site)
        S_site_sqrt = np.diag(np.sqrt(tau_site))
        L = cho_factor(JITTER * np.eye(N) + S_site_sqrt.dot(K).dot(S_site_sqrt), lower=True, check_finite=True)
        V = cho_solve(L, S_site_sqrt.dot(K))
        
        Sig = K - V.T.dot(V)
        mu_proposed = Sig.dot(nu_site)
        
        # Check convergence condition
        if la_norm(mu_proposed, mu) < tol:
            converged = True
            break
    
    # Compute marginal likelihood log evidence
    T = np.diag(tau_cav)
    
    one = np.sum( np.log( np.diagonal(L[0]) ) )
    two = 0.5 * nu_site.dot( Sig - np.diag( (tau_cav + tau_site)**(-1) ) ).dot(nu_site)
    three = np.sum( np.log( norm.cdf( y.dot( nu_cav / tau_cav ) / ( np.sqrt( 1 + tau_cav**(-1) ) ) ) ) )
    four = 0.5 * np.log( 1 + tau_site.dot( tau_cav**(-1) ) )
    five = 0.5 * (nu_cav / tau_cav).dot(T).dot( np.diag( (tau_cav + tau_site)**(-1) ) ).dot( S.dot(nu_cav / tau_cav) - 2 * nu_site)
    
    log_evidence = one + two + three + four + five
    return nu_site, v_site, log_evidence, converged
        
    

In [155]:
expectation_propagation_probit(y, gram)

2.0
-0.3447069853790401
4.0932499042962
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


/tmp/ipykernel_16966/3091254423.py:32: RuntimeWarning: invalid value encountered in sqrt
  mu_hat = ( nu_cav[i] / tau_cav[i] ) + ( y[i] * tau_cav[i]**(-1) * norm.pdf(z[i]) ) / ( norm.cdf(z[i]) * np.sqrt( 1 + tau_cav[i]**(-1) ) )
/tmp/ipykernel_16966/3091254423.py:47: RuntimeWarning: invalid value encountered in sqrt
  S_site_sqrt = np.diag(np.sqrt(tau_site))


ValueError: array must not contain infs or NaNs